In [ ]:
from google.colab import files
uploaded=files.upload()

In [21]:
import zipfile 
import io
data=zipfile.ZipFile(io.BytesIO(uploaded['Hojas IT.zip' ]),'r')
data.extractall() 

In [1]:
import numpy as np
import glob
import cv2

In [ ]:
# A Escala de Grises

In [ ]:
img = cv2.imread('./../Imagenes/Hojas/enfermas/Training/byZwaWQ9QXBp.jpg')
def getFeatures(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    threshold,_ = cv2.threshold(gray,0,255,cv2.THRESH_OTSU)
    mask = np.uint8(1*(gray<threshold))
    B=(1/255)*np.sum(img[:,:,0]*mask)/np.sum(mask)
    G=(1/255)*np.sum(img[:,:,1]*mask)/np.sum(mask)
    R=(1/255)*np.sum(img[:,:,2]*mask)/np.sum(mask)
    return [B,G,R]


In [ ]:
#Rutas para cargar las imagenes, Etiquetas y Caracteristicas 

In [ ]:
paths=['./../Imagenes/Hojas/enfermas/Training/',
'./../Imagenes/Hojas/saludables/Training/'
]
labels=[]
features=[]

In [ ]:
#Lectura a cada una de las imagenes para etiqutarlas y extraer sus caracteristicas

In [ ]:
for label, path in enumerate(paths):
    for path in paths :
        for filename in glob.glob(path+'*.jpg'):
            img=cv2.imread(filename)
            feature_vector=np.array(getFeatures(img))
            x=np
            features.append(x)
            labels.append(label)


In [ ]:
# Para el grafico de puntos en 3D

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Creacion de la grafica , los datos provenientes de features y labels 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, feature_row in enumerate(features):
    if labels[i] == -1:
        ax.scatter(feature_row[0], feature_row[1], feature_row[2], marker='*', c='k')

    else:
        #ERROR EN ESTA LINEA
      ax.scatter(feature_row[0],feature_row[1],feature_row[2], marker ='*',c='r')
        
ax.set_xlabel('B')
ax.set_ylabel('G')
ax.set_zlabel('R')

subFeatures = features[:,1::]
loss = []

In [ ]:
# Calculo de la distancia euclidiana entre los puntos de la grafica de puntos en 3R , al final los errores obtenidos se almacenan en la variable loss

In [ ]:
for w1 in np.linspace(-6, 6, 100):
    for w2 in np.linspace(-6, 6, 100):
        totalError = 0

        for i, feature_row in enumerate(subFeatures):
            sample_error = (w1 * feature_row[0] + w2 * feature_row[1] - labels[i]) ** 2
            
            totalError += sample_error
            
        loss.append([w1, w2, totalError])
        
loss = np.array(loss)

In [ ]:
# Creamos el grafico 3D

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')
ax1.plot_trisurf(loss[:,0], loss[:,1], loss[:,2], cmap=cm.jet, linewidth=0)
ax1.set_xlabel('w1')
ax1.set_ylabel('w2')
ax1.set_zlabel('loss')

In [ ]:
# Ahora se crean 2 matrices para calcular los coeficientes de la recta de regresion lineal utilizando la formula de minimos cuadrados

In [ ]:
A = np.zeros((4,4))
b = np.zeros((4,1))

for i, feature_row in enumerate(features):
    x = np.append([1], feature_row)
    x = x.reshape((4,1))
    y = labels[i]
    
    A = A + x * x.T
    b = b + x * y
    
invA = np.linalg.inv(A)
W = np.dot(invA, b)

X = np.arange(0, 1, 0.1)
Y = np.arange(0, 1, 0.1)
X, Y = np.meshgrid(X, Y)
Z = -(W[1] * X + W[2] * Y + W[0]) / W[3]


In [ ]:

import matplotlib.cm as cm

In [ ]:
# Regresion lineal para hace predicciones sobre cada imagen de la carpeta de prueba utilizando los valores de W

In [ ]:
ax.plot_surface(X, Y, Z, cmap=cm.Blues)

prediction = 1 * (W[0] + np.dot(features, W[1::])) >= 0
prediction = 2 * prediction - 1
error = np.sum(prediction != labels.reshape(-1,1)) / len(labels)
efectividad = 1 - error

RUTA_IMAGEN = './../Imagenes/Hojas/saludables/Training/eCZwaWQ9QXBp2.jpg'
img = cv2.imread(RUTA_IMAGEN)

feature_vector = np.array(getFeatures(img))
result = np.sign(W[0] + np.dot(feature_vector, W[1::]))

In [ ]:
# Condicional para saber si la imagen pertenece a una Hoja saludable o enferma

In [ ]:
if result == -1:
    print("Hoja ENFERMA")
else:
    print("Hoja SALUDABLE")